# Preprocessing pipeline

Procesamos los escenarios de test

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession

RESULT_PATH = "/Users/maria/Dropbox/UCM/PD2/final_scenarios/answers_empty.csv"
MODEL_PATH = 
df_results = pd.read_csv(RESULT_PATH)

# Cargamos el modelo entrenado 
model = load_model(MODEL_PATH)  # Cambia "model_path" por la ruta real del modelo

for i in range(0, 194):
    file = f"scenario_{fillz(i,3)}"
    # Leemos el dataframe con el escenario ya preprocesado con el formato del decoder
    df = spark.read.parquet(file)
    # Aplicamos el pipeline de procesamiento
    df_model_ready = 
    # Añadimos los datos meteorológicos
    df_con_meteo = 
    # Sacamos las predicciones con el modelo
    predictions = model.predict(df_model_ready)
    # Seleccionamos la fila con la predicción correspondiente para ese escenario
    pred = 
    # Rellenamos la información en el dataframe de resultados
    df_results.loc[df_results['scenario'] == file, 'pred'] = pred

# Guardamos el dataframe de resultados
df_results.to_csv(RESULT_PATH, index=False) 


# Predictions

Importamos el modelo final de la fase de entrenamiento

Realizamos las predicciones sobre el escenario procesado

Guardamos la predicción correspondiente en el CSV de respuestas